In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Crohns_Disease"
cohort = "GSE186582"

# Input paths
in_trait_dir = "../../input/GEO/Crohns_Disease"
in_cohort_dir = "../../input/GEO/Crohns_Disease/GSE186582"

# Output paths
out_data_file = "../../output/preprocess/Crohns_Disease/GSE186582.csv"
out_gene_data_file = "../../output/preprocess/Crohns_Disease/gene_data/GSE186582.csv"
out_clinical_data_file = "../../output/preprocess/Crohns_Disease/clinical_data/GSE186582.csv"
json_path = "../../output/preprocess/Crohns_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Expression data from intestinal mucosa of patients with Crohn disease"
!Series_summary	"We used microarrays to detail the global signature of gene expression underlying endoscopic recurrence of CD and identified distinct gene signature predicting postoperative recurrence."
!Series_overall_design	"Ileal samples from Crohn's disease patients and healthy samples from non-inflammatory controls were collected for RNA extraction and hybridization on Affymetrix microarrays. Inclusion criteria were age >18 years, ileal or ileocolonic CD and indication of CD‐related intestinal surgery. Endoscopic recurrence was defined by the presence of any ulcerated lesions at the anastomosis and/or on the neo-terminal ileum (Rutgeerts score > i0). Five hundred and twenty samples (520) were collected from the inflamed ileum (M0I) and the ileal margin (M0M) at time of surgery, and during post-operative endoscopy six month later (M6). We also collected 25 ileal non-IBD con

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import numpy as np
import os
import json
from typing import Optional, Callable, Dict, Any, List

# 1. Gene Expression Data Availability
# Based on the Series_summary and Series_overall_design, this dataset contains gene expression data
# from microarrays (Affymetrix), which is suitable for our analysis.
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability

# For trait (Crohn's Disease):
# The 'rutgeertrec' at key 5 indicates recurrence status which can be used for Crohn's Disease status
# Values: 'Rem' (Remission), 'Rec' (Recurrence), 'Ctrl' (Control subjects)
trait_row = 5

# For gender:
# Gender information is available at key 1
gender_row = 1

# For age:
# Age information is not available in the data
age_row = None

# 2.2 Data Type Conversion Functions

def convert_trait(value: str) -> int:
    """
    Convert trait value to binary. 
    Ctrl (control/healthy) = 0, Rem/Rec (Crohn's disease variants) = 1
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary: Control=0, Recurrence/Remission=1 (both are Crohn's Disease)
    if value == "Ctrl":
        return 0
    elif value in ["Rem", "Rec"]:
        return 1
    else:
        return None

def convert_gender(value: str) -> int:
    """
    Convert gender to binary. Female = 0, Male = 1
    """
    if value is None:
        return None
    
    # Extract value after colon if present
    if ":" in value:
        value = value.split(":", 1)[1].strip()
    
    # Convert to binary
    if value.lower() == "female":
        return 0
    elif value.lower() == "male":
        return 1
    else:
        return None

def convert_age(value: str) -> Optional[float]:
    """
    Convert age value to float.
    This function is defined for completeness but won't be used as age data is unavailable.
    """
    return None

# 3. Save Metadata - Initial Filtering
# Check if trait data is available (trait_row is not None)
is_trait_available = trait_row is not None
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Since we're only doing analysis in this step and not extracting features yet,
# we'll stop here. The clinical feature extraction will be handled in a later step
# when we have the appropriate data structures.

print(f"Data Analysis Results:")
print(f"Gene Expression Available: {is_gene_available}")
print(f"Trait Data Available: {is_trait_available}")
print(f"Trait Row: {trait_row}")
print(f"Gender Row: {gender_row}")
print(f"Age Row: {age_row}")


Data Analysis Results:
Gene Expression Available: True
Trait Data Available: True
Trait Row: 5
Gender Row: 1
Age Row: None


### Step 3: Gene Data Extraction

In [4]:
# 1. Re-identify the SOFT and matrix files to ensure we have the correct paths
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract the gene expression data from the matrix file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers)
print("\nFirst 20 gene/probe identifiers:")
print(gene_data.index[:20])

# 4. Print the dimensions of the gene expression data
print(f"\nGene data dimensions: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")

# Note: we keep is_gene_available as True since we successfully extracted gene expression data
is_gene_available = True



First 20 gene/probe identifiers:
Index(['1053_at', '121_at', '1316_at', '1405_i_at', '1487_at', '1552256_a_at',
       '1552257_a_at', '1552258_at', '1552266_at', '1552269_at',
       '1552272_a_at', '1552274_at', '1552277_a_at', '1552280_at',
       '1552281_at', '1552286_at', '1552287_s_at', '1552289_a_at',
       '1552293_at', '1552296_at'],
      dtype='object', name='ID')

Gene data dimensions: 20186 genes × 489 samples


### Step 4: Gene Identifier Review

In [5]:
# The identifiers appear to be Affymetrix probe IDs (e.g., "1053_at", "121_at") rather than standard human gene symbols
# These require mapping to standard gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene annotation data from the SOFT file
gene_annotation = get_gene_annotation(soft_file)

# 3. Preview the gene annotation dataframe
print("Gene annotation dataframe column names:")
print(gene_annotation.columns)

# Preview the first few rows to understand the data structure
print("\nPreview of gene annotation data:")
annotation_preview = preview_df(gene_annotation, n=3)
print(annotation_preview)

# Maintain gene availability status as True based on previous steps
is_gene_available = True


Gene annotation dataframe column names:
Index(['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date',
       'Sequence Type', 'Sequence Source', 'Target Description',
       'Representative Public ID', 'Gene Title', 'Gene Symbol',
       'ENTREZ_GENE_ID', 'RefSeq Transcript ID',
       'Gene Ontology Biological Process', 'Gene Ontology Cellular Component',
       'Gene Ontology Molecular Function'],
      dtype='object')

Preview of gene annotation data:
{'ID': ['1007_s_at', '1053_at', '117_at'], 'GB_ACC': ['U48705', 'M87338', 'X51757'], 'SPOT_ID': [nan, nan, nan], 'Species Scientific Name': ['Homo sapiens', 'Homo sapiens', 'Homo sapiens'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Exemplar sequence', 'Exemplar sequence', 'Exemplar sequence'], 'Sequence Source': ['Affymetrix Proprietary Database', 'GenBank', 'Affymetrix Proprietary Database'], 'Target Description': ['U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Human receptor ty

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain the probe IDs and gene symbols
# From examining the data:
# - 'ID' column contains probe identifiers (e.g., '1007_s_at', '1053_at')
# - 'Gene Symbol' column contains the gene symbols (e.g., 'DDR1 /// MIR4640', 'RFC2')

# 2. Create a gene mapping dataframe
gene_mapping = get_gene_mapping(gene_annotation, 'ID', 'Gene Symbol')

# 3. Apply the gene mapping to convert probe measurements to gene expression data
gene_data = apply_gene_mapping(gene_data, gene_mapping)

# Print some information about the results
print(f"Number of unique genes after mapping: {len(gene_data.index)}")
print(f"First 10 gene symbols: {gene_data.index[:10].tolist()}")
print(f"Gene expression data shape after mapping: {gene_data.shape[0]} genes × {gene_data.shape[1]} samples")


Number of unique genes after mapping: 18409
First 10 gene symbols: ['A1BG', 'A1BG-AS1', 'A1CF', 'A2M', 'A2M-AS1', 'A2ML1', 'A2MP1', 'A4GALT', 'A4GNT', 'AA06']
Gene expression data shape after mapping: 18409 genes × 489 samples


### Step 7: Data Normalization and Linking

In [8]:
# 1. Check if gene data is available after mapping
if gene_data.shape[0] == 0:
    print("Error: Gene expression matrix is empty after mapping.")
    # Mark the dataset as not usable due to lack of gene expression data
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=False,  # No usable gene data
        is_trait_available=True,
        is_biased=True,
        df=pd.DataFrame(),
        note="Failed to map probe IDs to gene symbols. The annotation format may not be compatible with the extraction methods."
    )
    print("Dataset deemed not usable due to lack of gene expression data.")
else:
    # Only proceed with normalization if we have gene data
    print("Normalizing gene symbols...")
    gene_data_normalized = normalize_gene_symbols_in_index(gene_data)
    print(f"Gene data shape after normalization: {gene_data_normalized.shape[0]} genes × {gene_data_normalized.shape[1]} samples")

    # Save the normalized gene data
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    gene_data_normalized.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
    
    # Extract clinical features from the original data source
    print("Extracting clinical features from the original source...")
    # Get background information and clinical data again
    background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
    clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
    background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)
    
    # Extract clinical features
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    print("Extracted clinical features preview:")
    print(preview_df(selected_clinical_df))
    print(f"Clinical data shape: {selected_clinical_df.shape}")
    
    # Save the extracted clinical features
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical features saved to {out_clinical_data_file}")
    
    # Link clinical and genetic data
    print("Linking clinical and genetic data...")
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, gene_data_normalized)
    print(f"Linked data shape: {linked_data.shape}")
    
    # Check if the linked data has adequate data
    if linked_data.shape[0] == 0 or linked_data.shape[1] <= 4:  # 4 is an arbitrary small number
        print("Error: Linked data has insufficient samples or features.")
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=True,
            is_biased=True,
            df=linked_data,
            note="Failed to properly link gene expression data with clinical features."
        )
        print("Dataset deemed not usable due to linking failure.")
    else:
        # Handle missing values systematically
        print("Handling missing values...")
        linked_data_clean = handle_missing_values(linked_data, trait_col=trait)
        print(f"Data shape after handling missing values: {linked_data_clean.shape}")
        
        # Check if there are still samples after missing value handling
        if linked_data_clean.shape[0] == 0:
            print("Error: No samples remain after handling missing values.")
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=True,
                df=pd.DataFrame(),
                note="All samples were removed during missing value handling."
            )
            print("Dataset deemed not usable as all samples were filtered out.")
        else:
            # Check if the dataset is biased
            print("\nChecking for bias in feature variables:")
            is_biased, linked_data_final = judge_and_remove_biased_features(linked_data_clean, trait)
            
            # Conduct final quality validation
            is_usable = validate_and_save_cohort_info(
                is_final=True,
                cohort=cohort,
                info_path=json_path,
                is_gene_available=True,
                is_trait_available=True,
                is_biased=is_biased,
                df=linked_data_final,
                note="Dataset contains gene expression data for Crohn's Disease patients, examining response to Infliximab treatment."
            )
            
            # Save linked data if usable
            if is_usable:
                os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
                linked_data_final.to_csv(out_data_file)
                print(f"Linked data saved to {out_data_file}")
                print(f"Final dataset shape: {linked_data_final.shape}")
            else:
                print("Dataset deemed not usable for trait association studies, linked data not saved.")

Normalizing gene symbols...
Gene data shape after normalization: 18122 genes × 489 samples


Normalized gene expression data saved to ../../output/preprocess/Crohns_Disease/gene_data/GSE186582.csv
Extracting clinical features from the original source...


Extracted clinical features preview:
{'GSM5656170': [1.0, 0.0], 'GSM5656171': [1.0, 0.0], 'GSM5656172': [1.0, 0.0], 'GSM5656173': [1.0, 0.0], 'GSM5656174': [1.0, 0.0], 'GSM5656175': [1.0, 1.0], 'GSM5656176': [1.0, 1.0], 'GSM5656177': [1.0, 1.0], 'GSM5656178': [1.0, 1.0], 'GSM5656179': [1.0, 1.0], 'GSM5656180': [0.0, 1.0], 'GSM5656181': [1.0, 0.0], 'GSM5656182': [1.0, 0.0], 'GSM5656183': [1.0, 0.0], 'GSM5656184': [1.0, 1.0], 'GSM5656185': [0.0, 1.0], 'GSM5656186': [1.0, 1.0], 'GSM5656187': [1.0, 1.0], 'GSM5656188': [1.0, 1.0], 'GSM5656189': [0.0, 0.0], 'GSM5656190': [0.0, 1.0], 'GSM5656191': [1.0, 1.0], 'GSM5656192': [1.0, 1.0], 'GSM5656193': [1.0, 0.0], 'GSM5656194': [1.0, 0.0], 'GSM5656195': [1.0, 0.0], 'GSM5656196': [1.0, 0.0], 'GSM5656197': [1.0, 0.0], 'GSM5656198': [1.0, 0.0], 'GSM5656199': [1.0, 0.0], 'GSM5656200': [1.0, 1.0], 'GSM5656201': [1.0, 1.0], 'GSM5656202': [0.0, 1.0], 'GSM5656203': [1.0, 1.0], 'GSM5656204': [1.0, 1.0], 'GSM5656205': [1.0, 1.0], 'GSM5656206': [1.0, 1.0], 

Data shape after handling missing values: (489, 18124)

Checking for bias in feature variables:
For the feature 'Crohns_Disease', the least common label is '0.0' with 25 occurrences. This represents 5.11% of the dataset.
The distribution of the feature 'Crohns_Disease' in this dataset is fine.

For the feature 'Gender', the least common label is '1.0' with 243 occurrences. This represents 49.69% of the dataset.
The distribution of the feature 'Gender' in this dataset is fine.



Linked data saved to ../../output/preprocess/Crohns_Disease/GSE186582.csv
Final dataset shape: (489, 18124)
